# Datos

Vamos a usar el dataset de IMDB para clasificación de reseñas de películas, el objetivo del mismo es detectar si una reseña tiene sentimiento **positivo** o **negativo**.

Descarguen el dataset de este [link](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews).

Y word2vec de este [link](https://drive.google.com/file/d/1XusPRjsCVcIdCQ2hQDDWcH_wayfn4nWb/view?usp=sharing).

-> Para correr esta notebook en colab suban los archivos a una carpeta **data** en la raiz de su drive personal.


In [1]:
from google.colab import drive
drive.mount("/content/drive")

! cp "/content/drive/My Drive/data/IMDB_Dataset.zip" .
! unzip -q IMDB_Dataset.zip
! rm IMDB_Dataset.zip
! ls

Mounted at /content/drive
 drive	'IMDB Dataset.csv'   sample_data   word2vec.txt


In [2]:
import pandas as pd
imdb_data = pd.read_csv("IMDB Dataset.csv")

#sentiment count
print(imdb_data.columns)
imdb_data['sentiment'].value_counts()

# Convert positive and negative into binary classes (1-0)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

sentiment_data = lb.fit_transform(imdb_data["sentiment"])
imdb_data['sentiment'] = sentiment_data

Index(['review', 'sentiment'], dtype='object')


In [3]:
imdb_data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


# Imports


In [8]:
import re
from bs4 import BeautifulSoup

import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\franz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\franz\AppData\Roaming\nltk_data...


True

# Preprocesamiento Inicial

Como toda tarea de NLP tenemos que comenzar preprocesando los datos, eliminando palabras que no nos sirve, caracteres especiales, etc.

En particular hay tres tareas a ser realizadas basadas en un análisis inicial del dataset (mirando ejemplos al azar del mismo)



1.   Eliminar tags html (vamos a utilizar BeautifulSoup para esto)
2.   Eliminar texto entre parentesis rectos (Usando la siguiente expresion regular: ```\[[^]]*\]``` )
3. Eliminar caracteres especiales, usando una regex quitar todos los caracteres que no son ni letras ni números (```[^a-zA-z0-9\s] ``` )



In [ ]:
def strip_html(text):
  pass

def remove_between_square_brackets(text):
  pass

def remove_special_characters(text):
  pass

def low_level_preproc(text):
  pass

#Apply function on review column
imdb_data['review'] = imdb_data['review'].apply(low_level_preproc)

# Preprocesamiento de alto nivel

Una vez tenemos el texto limpio y trabajable volvemos a hacer otro pasaje de preprocesamiento de más alto nivel, ahora vamos a querer:



1.   Transformar todo el texto a minúscula
2.   Quitar stop words (usando nltk)
3.   Lemmatizar usando nltk WordNetLemmatizer

Para todo esto vamos a necesitar trabajar con **tokens** palabras individuales, en este caso vamos a separar por **whitespace**, pero se podrían usar mejores estrategias.



In [ ]:
all_stopwords = set(stopwords.words("english"))

def remove_stop_words(full_text_line):
  pass

def lemmatize(text):
  pass

def high_level_preproc(text):
  pass

#Apply function on review column
imdb_data['review'] = imdb_data['review'].str.lower()
imdb_data['review'] = imdb_data['review'].apply(high_level_preproc)

In [ ]:
imdb_data['review'].head(10)

# Modelando

Para modelar vamos a comenzar separando el dataset.

In [ ]:
#split the dataset  
#train dataset
train_reviews = imdb_data.review[:40000]
train_sentiments = imdb_data.sentiment[:40000]

#test dataset
test_reviews = imdb_data.review[40000:]
test_sentiments = imdb_data.sentiment[40000:]


print("Train set:", train_reviews.shape, train_sentiments.shape)
print("Test set:", test_reviews.shape, test_sentiments.shape)

Vamos a generar vectores para las reseñas usando TF-IDF (sklearn). Vamos a hacer uso del parametro ```max_features``` que nos permite controlar cuántas palabras considerar para generar los vectores (en orden de frecuencia). Luego usamos esa representacion vectorial para entrenar y testear un regresor logístico (LogisticRegressor). En particular vamos a empezar con 300 features, más adelante veremos por qué.


Vamos a entrenar el modelo por 500 iteraciones como máximo y usamos l2 como regularizador.

In [ ]:
# Su código para vectorizar

In [ ]:
# Su código para el modelo

# Vectores pre entrenados
Ahora vamos a ver si podemos superar la performance del modelo haciendo uso de deep learning.  Primero vamos a entrenar el mismo modelo usando los embeddings preentrenados de Word2Vec (usando gensim). 

Luego vamos a darle esos embeddings a un MLP y ver si logramos superar la performance anterior.

In [ ]:
import gensim

In [ ]:
w2v = gensim.models.KeyedVectors.load_word2vec_format("word2vec.txt", binary=False)

In [ ]:
mean_vector = np.mean(w2v.vectors, axis=0)

In [ ]:
def get_sentence_embedding(text):
  pass


train_vectors = [get_sentence_embedding(sent) for sent in train_reviews]
test_vectors = [get_sentence_embedding(sent) for sent in test_reviews]

In [ ]:
# Su código para el modelo (Igual que antes)

# Deep Learning


MLP: vamos a crear un MLP para atacar ese mismo problema, el diseño corre por su cuenta pero deberían ser capaces de obetener mejor performance en test que los modelos anteriores.


In [ ]:
# Imports
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
def train_epoch(training_model, loader, criterion, optim):
    training_model.train()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []
    
    for data, labels in loader:
      all_labels.extend(labels.numpy())  

      optim.zero_grad()

      predictions = training_model(data.to(DEVICE))
      all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

      loss = criterion(predictions, labels.to(DEVICE))
      
      loss.backward()
      optim.step()

      epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100


def validation_epoch(val_model, loader, criterion):
    val_model.eval()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():
      for data, labels in loader:
        all_labels.extend(labels.numpy())  

        predictions = val_model(data.to(DEVICE))
        all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

        loss = criterion(predictions, labels.to(DEVICE))

        epoch_loss += loss.item()

    return epoch_loss / len(loader), accuracy_score(all_labels, all_predictions) * 100
  

def train_model(model, train_loader, test_loader, criterion, optim, number_epochs):
  train_history = []
  test_history = []
  accuracy_history = []

  for epoch in range(number_epochs):
      start_time = time.time()

      train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
      train_history.append(train_loss)
      print("Training epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, train_loss, train_acc, time.time() - start_time))

      start_time = time.time()
      test_loss, acc = validation_epoch(model, test_loader, criterion)
      test_history.append(test_loss)
      accuracy_history.append(acc)
      print("Validation epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, test_loss, acc, time.time() - start_time))

In [ ]:
class MLP(nn.Module):

  def __init__(self, in_features):
    super(MLP, self).__init__()
    # Su implementacion

  def forward(self, new_input):
    # Su implementacion
    pass

In [ ]:
loss_function = nn.CrossEntropyLoss().to(DEVICE)
modelo = MLP(in_features=300).to(DEVICE)
optimizer = torch.optim.Adam(modelo.parameters(), lr=0.001)
BATCH_SIZE = 32

In [ ]:
# Dataloaders
train_vectors = [get_sentence_embedding(sent) for sent in train_reviews]
test_vectors = [get_sentence_embedding(sent) for sent in test_reviews]

train_targets = torch.Tensor(train_sentiments.to_numpy()).long()
train_dataset = TensorDataset(torch.Tensor(train_vectors), train_targets) 
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=2)

test_targets = torch.Tensor(test_sentiments.to_numpy()).long()
test_dataset = TensorDataset(torch.Tensor(test_vectors), test_targets) 
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=2)

In [ ]:
train_model(modelo, train_dataloader, test_dataloader, loss_function, optimizer, 10)

# Exploración

Exploren otras técnicas de preprocesamiento, tokenizacion, vectorizacion, etc. para ver si puede lograr superar los modelos presentados en clase.
